In [1]:
import json

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np
import numpy.linalg as nla

init_notebook_mode(connected=True)

In [2]:
def plot_util_comp(given_utils, r):
    #TODO: Add back diameter 
    
    fig = go.Figure()

    sim_names = ['MWU', 'Even', 'MWU (Self-Edges)']
    lcolors = ['Blue', 'darkorange', 'darkseagreen']
    mcolors = ['LightSkyBlue', 'orange', 'limegreen']
    
    for idx, social_utils in enumerate(given_utils):
        name = sim_names[idx]
        mcolor = mcolors[idx]
        lcolor = lcolors[idx]
        
        #Add averages trace
        p_vals = [ p for (p, uts) in social_utils ]
        ut_vals = [ np.average(uts) for (p, uts) in social_utils ]
        fig.add_trace(go.Scatter(x=p_vals,
            y=ut_vals, mode='lines+markers', hoverinfo='skip',
            name=f'avg social welfare rate ({name})', marker=dict(color=lcolor)))

        #Add individual points trace
        indiv_p_vals = []
        indiv_ut_vals = []
        for (p, uts) in social_utils:
            indiv_ut_vals.extend(uts)
            indiv_p_vals.extend([p] * len(uts))
        fig.add_trace(go.Scatter(x=indiv_p_vals,
            y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
            marker=dict(color=mcolor), name=f'social welfare per p ({name})'))
    
    fig.layout.update(showlegend=False)
    title_fmt = "Average Social Welfare (Normalized) vs p (r={0})"
    plot_title = title_fmt.format(r)
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='avg social welfare'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [3]:
def plot_marginal_gains(mg_means, r):
    fig = go.Figure()
    
    mg_names = ['MWU', 'Even', 'MWU (Self-Edges)']
    lcolors = ['Blue', 'darkorange', 'darkseagreen']
    mcolors = ['LightSkyBlue', 'orange', 'limegreen']
    
    for idx, mg_vals in enumerate(mg_means):
        name = mg_names[idx]
        lcol = lcolors[idx]
        mcol = mcolors[idx]
        
        #Add averages trace
        p_vals = [ p for (p, mg) in mg_vals ]
        mg_avgs = [ np.average(mg) for (p, mg) in mg_vals ]
        fig.add_trace(go.Scatter(x=p_vals,
            y=mg_avgs, mode='lines+markers', hoverinfo='skip',
            name=f'avg {name} distance ratio', marker=dict(color=lcol)))

        #Add individual points trace
        indiv_p_vals = []
        indiv_mg_vals = []
        for (p, mg) in mg_vals:
            indiv_mg_vals.extend(mg)
            indiv_p_vals.extend([p] * len(mg))
        fig.add_trace(go.Scatter(x=indiv_p_vals,
            y=indiv_mg_vals, mode='markers', hoverinfo='skip', opacity=0.7,
            marker=dict(color=mcol), name=f'{name} marginal gains'))

    fig.layout.update(showlegend=False)
    title_fmt = "Marginal Gains vs p (r={0})"
    plot_title = title_fmt.format(r)
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='Avg Marginal Gain'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [4]:
def plot_data(data):
    plaw = data['plaw']
    
    for r_val, p_data in data.items():
        if r_val == 'plaw':
            continue
        
        #Data to be plotted
        mwu_utils = []
        even_utils = []
        se_mwu_utils = []
        
        mwu_mgs = []
        even_mgs = []
        se_mwu_mgs = []
            
        for p_val, sim_data in p_data.items():
            full_utils = sim_data['utils']
            full_maps = sim_data['maps']
            
            final_ut = lambda uts : [ sum(ut[-1]) / len(ut[-1]) for ut in uts ]
            
            def get_marginal_gain(key):
                mg_list = []
                for idx, ut_map in enumerate(full_maps[key]):
                    init_ut = full_utils[key][idx][0]

                    run_mg = []
                    for vtx, vum in ut_map.items():
                        gain = vum['ut'] - init_ut[int(vtx)]
                        interactions = vum['int_cnt']

                        if interactions == 0:
                            continue

                        run_mg.append(gain / interactions)
                    mg_list.append(np.average(run_mg))
                return mg_list
            
            def add_sim_data(key, util_list, mgs_list):
                utils = final_ut(full_utils[key])
                util_list.append((p_val, utils))
                
                mgs = get_marginal_gain(key)
                mgs_list.append((p_val, mgs))
                
            add_sim_data('mwu', mwu_utils, mwu_mgs)
            add_sim_data('even', even_utils, even_mgs)
            add_sim_data('se_mwu', se_mwu_utils, se_mwu_mgs)
            
        plot_util_comp([mwu_utils, even_utils, se_mwu_utils], r_val)
        plot_marginal_gains([mwu_mgs, even_mgs, se_mwu_mgs], r_val)

In [5]:
def plot_sim_results(plaw=False):
    if plaw:
        kgrid_plaw_data = None
        with open('data/kgrid_plaw_data.json', 'r') as kpd:
            kgrid_plaw_data = json.loads(kpd.read())
        plot_data(kgrid_plaw_data)
    else:
        kgrid_data = None
        with open('data/kgrid_data.json', 'r') as kd:
            kgrid_data = json.loads(kd.read())
        plot_data(kgrid_data)

## Marginal Gains

In [6]:
plot_sim_results()

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:356: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning:

invalid value encountered in double_scalars



### Powerlaw Distributed Resources

In [7]:
plot_sim_results(True)